# Experiment 1

## Modules and settings

In [ ]:
import numpy as np
import pandas as pd
import data_cleaning as dc
import matplotlib.pyplot as plt
import scipy.stats as sts
import scipy.optimize as opt
import seaborn as sns
A, B, C, D = sns.color_palette('muted', 4)
sns.set_style('white')

## Directories

In [ ]:
data_source = "./Raw/"
data_out = "./Clean/"
figure_out = "./Figures/weiji_talk/"

# Data Processing

In [ ]:
def detect_error(row):
    is_error = np.nan
    if row.status == 'AFC2':
        if ((row.a_val > row.b_val) & (row.response == row.b)):
            is_error = 1
        elif ((row.a_val < row.b_val) & (row.response == row.a)):
            is_error = 1 
        elif row.a_val == row.b_val:
            is_error = .5
        else:
            is_error = 0
    else:
        is_error = int(row.gtv < row.posqual)
    return is_error

## Load data

In [ ]:
subject_initials = ["AL", "AW", "EC", "FC", "GA", 
                    "HW", "JH", "JZ", "LH", "MH", "MS", "ZL"] #"SVO", "GVG"]
files = [data_source + s + ".csv" for s in subject_initials]
subjects = [dc.clean(f) for f in files]

for i in np.arange(len(subjects)):
    subjects[i] = subjects[i].loc[
        subjects[i].human == subjects[i].subject, :].reset_index(drop=True)
    subjects[i].loc[:, 'subject'] = str(i)
    subjects[i].loc[:, 'subject'] = subjects[i].loc[:, 'subject'].astype(int)
    
data = pd.concat(subjects).reset_index(drop=True)
data = data.reindex_axis(["subject","color","gi","mi","status",
                          "bp","wp","response","rt",'human','computer', 'time'], axis=1)

boards = pd.read_csv('experiment_boards_new.txt', sep='\t', 
                     names=['bp', 'wp', 'a', 'a_val', 'b', 'b_val', 'color', 'muinf'])
boards.loc[:, 'color'] = boards.loc[:, 'color'].map({'BLACK':0, 'WHITE':1})

objlists = ['objective_values_gen2.txt', 
            'objective_values_hvh.txt', 
            'objective_values_eye.txt']

obj = pd.concat([pd.read_csv(data_out + o, sep='\t', 
                            names=['bp', 'wp'] + list(range(36))) 
                for o in objlists]).reset_index(drop=True)

dupecols = ['subject', 'status', 'color', 'mi', 'bp', 'wp', 'response']
ai = data.loc[~data.status.isin(['AFC2', 'EVAL']), :]
afc = data.loc[data.status=='AFC2', :].drop_duplicates(subset=dupecols)
eva = data.loc[data.status=='EVAL', :].drop_duplicates(subset=dupecols)
data = pd.concat([ai, afc, eva]).reset_index(drop=True)

In [ ]:
data.loc[:, 'gtv'] = np.nan
data.loc[:, 'posqual'] = np.nan
data.loc[:, 'error'] = np.nan
data.loc[:, 'a'] = np.nan
data.loc[:, 'b'] = np.nan

for i in data.index.values:
    r = int(data.loc[i, 'response'])
    bp = data.loc[i, 'bp']
    wp = data.loc[i, 'wp']
    c = data.loc[i, 'color']
    
    match = obj.loc[(obj.bp == bp) & (obj.wp == wp), :]
    bmatch = boards.loc[(boards.bp == bp) & (boards.wp == wp), :]
    
    if len(match) >= 1:
        gtv = match.loc[:, 35-r].values[0]
        gtv = gtv if ((gtv==0) | (np.abs(gtv) == 10000)) else 0
        gtv = gtv/10000
        pqual = np.nanmax(
            match.loc[:, list(range(35))].values[0]) if c == 0 else np.nanmin(
            match.loc[:, list(range(35))].values[0])
        pqual = pqual if ((pqual==0) | (np.abs(pqual) == 10000)) else 0
        pqual = pqual/10000
        data.loc[i, 'gtv'] = gtv if ((c == 0) | (gtv==0)) else -gtv
        data.loc[i, 'posqual'] = pqual if ((c == 0) | (pqual==0)) else -pqual
        
        if len(bmatch) >= 1:
            a = bmatch.loc[:, 'a'].values[0]
            b = bmatch.loc[:, 'b'].values[0]
            a_val = match.loc[:, 35-a].values[0]
            a_val = a_val if ((a_val==0) | (np.abs(a_val)==10000)) else 0
            a_val /= 10000
            
            b_val = match.loc[:, 35-b].values[0]
            b_val = b_val if ((b_val==0) | (np.abs(b_val)==10000)) else 0
            b_val /= 10000
            
            data.loc[i, 'a'] = a
            data.loc[i, 'b'] = b
            data.loc[i, 'a_val'] = a_val 
            data.loc[i, 'b_val'] = b_val
        data.loc[i, 'error'] = detect_error(data.loc[i, :])
        gtv = data.loc[i, 'gtv']
        pqual = data.loc[i, 'posqual']
        data.loc[i, 'gtv'] = gtv if ((c == 0) | (gtv==0)) else -gtv
        data.loc[i, 'posqual'] = pqual if ((c == 0) | (pqual==0)) else -pqual

# data.loc[data.status=='AFC2', 'error'] = 1 - data.loc[data.status=='AFC2', 'error']        

# data.loc[data.status=='AFC2', :].to_csv(data_out + 'afc2.csv')
# data.loc[data.status=='EVAL', ['subject', 'color', 'gi', 'mi', 'bp', 'wp', 'response', 'rt', 'human', 'computer', 'time', 'gtv']].to_csv(data_out + 'eval.csv')
# data.loc[~data.status.isin(['AFC2', 'EVAL']), :].to_csv(data_out + 'ai.csv')
data.head()


In [ ]:
data.status.unique()

In [ ]:
len(data.loc[data.status == 'AFC2', :])

In [ ]:
impcols = ['subject', 'color', 'gi', 'mi', 'status',
                        'bp', 'wp', 'response', 'rt', 'human', 
                        'computer', 'time', 'a', 'b', 'a_val', 'b_val']

data = pd.read_csv(data_out + 'afc2 2.csv', names=impcols)

for i in data.index.values:
    r = int(data.loc[i, 'response'])
    bp = data.loc[i, 'bp']
    wp = data.loc[i, 'wp']
    c = data.loc[i, 'color']
    
    match = obj.loc[(obj.bp == bp) & (obj.wp == wp), :]
    bmatch = boards.loc[(boards.bp == bp) & (boards.wp == wp), :]
    
    if len(match) >= 1:
        gtv = match.loc[:, 35-r].values[0]
        gtv = gtv if ((gtv==0) | (np.abs(gtv) == 10000)) else 0
        gtv = gtv/10000
        pqual = np.nanmax(
            match.loc[:, list(range(35))].values[0]) if c == 0 else np.nanmin(
            match.loc[:, list(range(35))].values[0])
        pqual = pqual if ((pqual==0) | (np.abs(pqual) == 10000)) else 0
        pqual = pqual/10000
        data.loc[i, 'gtv'] = gtv if ((c == 0) | (gtv==0)) else -gtv
        data.loc[i, 'posqual'] = pqual if ((c == 0) | (pqual==0)) else -pqual
        
        if len(bmatch) >= 1:
            a = bmatch.loc[:, 'a'].values[0]
            b = bmatch.loc[:, 'b'].values[0]
            a_val = match.loc[:, 35-a].values[0]
            a_val = a_val if ((a_val==0) | (np.abs(a_val)==10000)) else 0
            a_val /= 10000
            
            b_val = match.loc[:, 35-b].values[0]
            b_val = b_val if ((b_val==0) | (np.abs(b_val)==10000)) else 0
            b_val /= 10000
            
            data.loc[i, 'a'] = a
            data.loc[i, 'b'] = b
            data.loc[i, 'a_val'] = a_val
            data.loc[i, 'b_val'] = b_val
        data.loc[i, 'error'] = detect_error(data.loc[i, :])
        
        av = data.loc[i, 'a_val']
        bv = data.loc[i, 'b_val']
        data.loc[i, 'a_val'] =  av if ((c == 0) | (gtv==0)) else -av
        data.loc[i, 'b_val'] = bv if ((c == 0) | (gtv==0)) else -bv

data.loc[data.status=='AFC2', 'error'] = 1 - data.loc[data.status=='AFC2', 'error']

data.loc[:, ['subject', 'color', 'gi', 'mi', 'status',
                        'bp', 'wp', 'response', 'rt', 'human', 
                        'computer', 'time', 'a', 'b', 
             'a_val', 'b_val']].reset_index(drop=True).to_csv(data_out + 'afc2_new.csv', 
                                                              header=False)
data

In [ ]:
data.loc[data.status=='AFC2']


## Analyses

In [ ]:
sns.set_palette('deep')
sns.set_style('white')
sns.set_context('poster')
B, G, R, P = sns.color_palette('deep', 4)
%matplotlib inline

### AI

#### Select Data

In [ ]:
# data.loc[data.status=='AFC2', 'color'] = (data.loc[data.status=='AFC2', 'color'] + 1) % 2

probe = np.array([np.array(list(data.loc[i, 'bp'])).astype(int).sum() - 
          np.array(list(data.loc[i, 'wp'])).astype(int).sum() 
         for i in data.index.values])


data.loc[probe != data.color, :]

In [ ]:
d = data.loc[~data.status.isin(['AFC2', 'EVAL']), :].reset_index(drop=True)
for dex in d.loc[(d.mi==0)&(d.gi==0), :].index.values:
    d.loc[dex, 'computer'] = d.loc[dex+1, 'computer']

In [ ]:
# wins n draws
dex = d.loc[d.mi.isin([0,1]), :].index.values
ends = d.loc[dex[1:]-1, :]
ends.loc[ends.status=='playing', 'status'] = 'loss'
ends.reset_index(drop=True).loc[:, ['subject', 'computer', 'gi', 'color', 'status']].to_csv(data_out + "game_ends_for_elo.csv", index=False, header=False)
ends
ends.loc[:, 'wld'] = ends.loc[:, 'status'].map({'loss':0, 'draw':.5, 'win':1})
def compasint(i):
    return int(float(i) - .01)
# (ends.computer.unique().astype(float) - .01).astype(int)
ends.loc[:, 'computer'] = ends.loc[:, 'computer'].map(compasint)
ends.loc[:, 'tier'] = ends.loc[:, 'computer'] // 5

In [ ]:
games_per_computer = pd.DataFrame(index=ends.subject.unique(), columns=np.arange(30))
games_per_computer.loc[:, :] = 0

wins_per_computer = pd.DataFrame(index=ends.subject.unique(), columns=np.arange(30))
wins_per_computer.loc[:, :] = 0

draws_per_computer = pd.DataFrame(index=ends.subject.unique(), columns=np.arange(30))
draws_per_computer.loc[:, :] = 0


endsnodraw = ends.loc[ends.status!='draw', :]
endsdraw = ends.loc[ends.status=='draw', :]

for s in endsnodraw.index.values:
    wins_per_computer.loc[endsnodraw.loc[s, 'subject'], endsnodraw.loc[s, 'computer']] += endsnodraw.loc[s, 'wld']
    
for s in ends.index.values:
    games_per_computer.loc[ends.loc[s, 'subject'], ends.loc[s, 'computer']] += 1
    
for s in endsdraw.index.values:
    draws_per_computer.loc[endsdraw.loc[s, 'subject'], endsdraw.loc[s, 'computer']] += endsdraw.loc[s, 'wld']*2
    
games_per_computer.to_csv(data_out + 'games_x_computer.csv', header=False, index=False)
wins_per_computer.to_csv(data_out + 'wins_x_computer.csv', header=False, index=False)
draws_per_computer.to_csv(data_out + 'draws_x_computer.csv', header=False, index=False)

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(20,6), squeeze=False)

dex = np.arange(30)
rate_means = []
rate_errs = []

for _d in dex:
    _e = ends.loc[(ends.computer==_d), 'wld'].values
#     print(_d, _e.size, np.mean(_e))
    m = np.mean(_e)
    if _e.size > 0:
        rate_means.append(m)
#         rate_errs[_d] = np.std(_e)
    else:
        rate_means.append(np.nan)
    
axes[0,0].plot(dex, rate_means, color='black')
axes[0,0].set_xlabel('Computer')

dex = np.arange(6)
tier_means = []

for _d in dex:
    _e = ends.loc[(ends.tier==_d), 'wld'].values
    m = np.mean(_e)
    if _e.size > 0:
        tier_means.append(m)
    else:
        tier_means.append(np.nan)
        
axes[0,1].plot(dex, tier_means, color='black')
axes[0,1].set_xlabel('Tier')
sns.despine()
f.savefig(figure_out + 'psychometric_curve_ai_all_subs.png', format='png')

# resample means

In [ ]:
f, axes = plt.subplots(1,1, figsize=(12,12), squeeze=False)
axes[0, 0].plot([-1, 6], [1, 1], '--', color='grey')
sns.regplot('tier', 'wld', data=ends, x_estimator=np.mean, fit_reg=True,
            logistic=True, n_boot=len(ends), color='black', ax=axes[0,0], 
            scatter_kws={'s':150, 'linewidth':2}, line_kws={'linewidth':3})
axes[0,0].set_ylim([0, 1.05])
axes[0, 0].set_ylabel('Performance')
axes[0,0].set_xlim([-.3, 5.3])
axes[0,0].set_xlabel('Tier')
sns.despine()
f.savefig(figure_out + 'psychometric_curve.png', format='png')

In [ ]:
# d = ai.loc[:, ['subject', 'color', 'bp', 'wp', 'response', 'computer', 'gi', 'mi', 'time']]
# fig, axes = plt.subplots(nrows=len(d.subject.unique()), ncols=1, figsize=(10,15), squeeze=False)

# for s in ai.subject.unique():
#     curax = axes[s, 0]
#     p = d.loc[(d.subject==s)&((d.mi==d.color)|(d.mi==d.color)), :]
#     curax.plot(p.time.values - p.time.values[0], p.computer.values//5, label='bracket')
#     curax.plot(p.time.values - p.time.values[0], p.computer.values/5, label='opponent')
#     curax.legend()
    
# # fig.savefig(figure_out + 'opponent_path.png', format='png')

In [ ]:
stdr[1]

In [ ]:
# what bas wants: at each time, average value of current opponent
d.loc[:, 'adjtime'] = np.concatenate([
        d.loc[d.subject==s, 'time'].values - 
        d.loc[d.subject==s, 'time'].values[0] for s in d.subject.unique()])

# time_bins1 = np.linspace(0, d.loc[:, 'adjtime'].max(), 101)
time_bins2 = np.linspace(0, d.loc[:, 'adjtime'].max(), 11)

_dd = d.sort('adjtime').reset_index(drop=True)

x2 = []
y2 = []
err2 = []

for t in range(len(time_bins2))[:-1]:
    interval = time_bins2[t:t+2]
    _ddd = _dd.loc[(_dd.adjtime >= interval[0]) & (_dd.adjtime < interval[1]), :]
    x2.append(t)
    y2.append((_ddd.loc[:, 'computer']//5).mean())
    meanr, varr, stdr = sts.bayes_mvs(_ddd.loc[:, 'computer']//5)
    err2.append(meanr[1])
# print(err2)
x2 = np.array(x2)
y2 = np.array(y2)

fig, axes = plt.subplots(1, 1, squeeze=False, figsize=(6,6))
ax = axes[0,0]
# ax.plot(x1, y1, color=P)
ax.plot(np.array(x2)*11, y2, color='black')
for er in range(len(err2)):
    ax.plot([11*x2[er], 11*x2[er]], [err2[er][0], err2[er][1]], color='black')
ax.set_xlabel('Time (% of total time)', fontsize=18)
ax.set_ylim([0,5])
ax.set_ylabel('Tier', fontsize=18)
sns.despine()
fig.savefig(figure_out + 'averaged_opponent.png', format='png')

## AFC

### Helper Functions

In [ ]:
def plt_agrees(df, ptype="agree", 
               ax=plt.gca(), fig=plt.gcf()):
    
    if ptype=="agree":
        val = (df.values.sum() - len(df))/df.values.size
        ax.set_title('Mean Agreement: ' + str(val)[:6])
    elif ptype=="dist":
        val = (df.values.sum() - .5*len(df)) / df.values.size
        ax.set_title('Mean distance From Random : ' + str(val)[:6])
    
    sns.heatmap(data=df, vmin=df.values.min(), vmax=df.values.max(), 
            annot=True, cbar=False, ax=ax, square=True);
        
    return fig, axes, val;

### Summary Table Of Agreements

In [ ]:
dd = data.loc[data.status=='AFC2', :]

for n in range(len(subjects)):
    dd.loc[:, n] = np.nan

for s1 in dd.subject.unique():
    
    for s2 in dd.subject.unique():
        
        for i in dd.loc[dd.subject == s1, :].index.values:
            bp = dd.loc[i, 'bp']
            wp = dd.loc[i, 'wp']
            r = dd.loc[i, 'response']
            condition = (dd.subject == s2) & (dd.bp == bp) & (dd.wp == wp)
           
            if len(dd.loc[condition, :]) == 1:
                if dd.loc[condition, 'response'].values[0] == r:
                    dd.loc[i, s2] = 1
                else:
                    dd.loc[i, s2] = 0

afc_by_subject = [dd.loc[dd.subject==s, :].reset_index(drop=True) 
                  for s in dd.subject.unique()]

ca_helper = pd.DataFrame(index=range(len(afc_by_subject)), 
                         columns=range(len(afc_by_subject)))

for _s1 in range(len(afc_by_subject)):
    s1 = afc_by_subject[_s1]
    for _s2 in range(len(afc_by_subject)):
        s2 = afc_by_subject[_s2]
        _ag = (s1.loc[:, _s1] == s1.loc[:, _s2]).astype(int)
        ca_helper.loc[_s1, _s2] = _ag.sum()/len(_ag)
                                   
ca_helper = ca_helper.astype(float)
ca_dist = np.abs(ca_helper - .5)

good_guys = ca_dist.loc[ca_dist.mean() >= ca_dist.values.mean(), :].index.values
good_guys_ag = ca_helper.loc[good_guys, good_guys]
good_guys_dist = ca_dist.loc[good_guys, good_guys]
okay_guys = ca_dist.loc[ca_dist.mean() >= ca_dist.values.mean()*.75, :].index.values
okay_ag = ca_helper.loc[okay_guys, okay_guys]
okay_d = ca_dist.loc[okay_guys, okay_guys]

In [ ]:
f, axes = plt.subplots(3, 2, figsize=(16, 30), squeeze=False)

plt_agrees(okay_d, ptype="dist", ax=axes[0,0]);
plt_agrees(okay_ag, ptype="agree", ax=axes[0,1]);
plt_agrees(good_guys_dist, ptype="dist", ax=axes[1,0]);
plt_agrees(good_guys_ag, ptype="agree", ax=axes[1,1]);
plt_agrees(ca_helper, ptype="agree", ax=axes[2,1]);
plt_agrees(ca_dist, ptype="dist", ax=axes[2,0]);
f.savefig(figure_out + 'agreement_histograms.png', format='png')

In [ ]:
data.loc[data.status=='AFC2', 'error'] = 1 - data.loc[data.status=='AFC2', 'error']
data.loc[:, 'error'] = 1 - data.loc[:, 'error']

In [ ]:
sns.set_style('white')
afc_perf = np.array([data.loc[(data.subject==s)&(data.status=='AFC2')&(data.error!=.5), 'error'].mean() 
                     for s in data.subject.unique()])
ai_perf = np.array([data.loc[(data.subject==s)&(~data.status.isin(['AFC2','EVAL'])), 'error'].mean() 
                    for s in data.subject.unique()])
ai_ELO = np.array([224, 138, -45, 0, 29, -9, -5,57, -330, 115, -72, 164])


f, axes = plt.subplots(1, 1, figsize=(6, 6), squeeze=False)
ax = axes[0,0]
ax.set_xlim([.75, 1])
ax.set_ylim([0, 1])
ax.set_xlabel('Performance In AI')
ax.set_ylabel('Performance in AFC')
sns.regplot(ai_perf, afc_perf, color='black', ax=ax, ci=False, line_kws={'linewidth':2})

# ax = axes[0, 1]
# ax.set_ylim([-400, 400])
# ax.set_xlim([.5, 1])
# ax.set_xlabel('Performance In AI')
# ax.set_ylabel('ELO')
# sns.regplot(ai_perf, ai_ELO, color='black', ax=ax, ci=False)

sns.despine()
# print(sts.pearsonr(ai_ELO, ai_perf))
ax.text(x=.9, y=.8, s="r = " + str(sts.pearsonr(ai_perf, afc_perf)[0])[:4], fontsize=18)
ax.text(x=.9, y=.73, s="p = " + str(sts.pearsonr(ai_perf, afc_perf)[1])[:4], fontsize=18)
# print(sts.pearsonr(ai_ELO, afc_perf))
f.savefig(figure_out + 'performance_afc_v_ai.png', format='png')

In [ ]:
sns.set_style('white')
f, axes = plt.subplots(1, 1, figsize=(7, 7), squeeze=False)
ax = axes[0, 0]
sns.barplot('subject', 'error', data=data.loc[(data.status=='AFC2')&(data.error != .5), :], ax=ax, color=R)
ax.set_ylim([0,1])
ax.set_xlabel('Subject')
ax.set_ylabel('Performance')

# ax = axes[0, 1]
# ax.set_ylim([-400, 400])
# ax.set_xlim([.5, 1])
# ax.set_xlabel('Performance In AI')
# ax.set_ylabel('ELO')
# m, b, r, p, s = sts.linregress(ai_perf, ai_ELO)
# ax.text(x=.8, y=300, s="r = " + str(r)[:4], fontsize=18)
# ax.text(x=.8, y=250, s="p = " + str(p)[:4], fontsize=18)
# sns.regplot(ai_perf, ai_ELO, color='black', ax=ax, ci=False, line_kws={'linewidth':2})

sns.despine()
f.savefig(figure_out + 'performance_by_subject.png', format='png')

In [ ]:
arf = data.loc[(data.status=='AFC2')&(data.error != .5), :]
arf.loc[:, 'wholeboard'] = arf.loc[:, 'bp'] + arf.loc[:, 'wp']

perf = np.array([np.nanmean(arf.loc[arf.wholeboard == b, 'error']) 
                 for b in arf.wholeboard.unique()])
np.sort(perf)
easy = arf.loc[arf.wholeboard == arf.wholeboard.unique()[perf.argmax()], :]
hard = arf.loc[arf.wholeboard == arf.wholeboard.unique()[perf.argmin()], :]
hard

In [ ]:
f, ax = plt.subplots(1,1,figsize=(10,8))
sns.distplot(perf, norm_hist=False, kde=False, bins=np.linspace(0, 1, 14), color='black')
ax.set_xticks([0, 1])
ax.set_xticklabels([0, 1], fontsize=32)
ax.set_yticklabels(list(range(8)), fontsize=32)
ax.set_ylabel('')
ax.set_xlabel('Performance Levels', fontsize=32)
sns.despine()
f.savefig(figure_out + 'performance level histogram.png', format='png')

In [ ]:
hard

## Evaluation

In [ ]:
def expandposition(string):
    _q = np.array(list(string)).astype(int)
    return np.where(_q==1)[0]

def plotpos(frame, axe):
    bp = expandposition(frame.loc[frame.index.values[0], 'bp'])
    wp = expandposition(frame.loc[frame.index.values[0], 'wp'])
    a = frame.loc[frame.index.values[0], 'a']
    b = frame.loc[frame.index.values[0], 'b']
    c = frame.loc[frame.index.values[0], 'color']
    if c==0:
        correct = a if a > b else b
    elif c==1:
        correct = a if a < b else b
    
    bpx = bp % 9
    bpy = bp // 9
    wpx = wp %9
    wpy = wp //9
    ax = a%9
    ay = a//9
    bx = b%9
    by = b//9
    cx = correct % 9
    cy = correct // 9
    
    axe.scatter(bpx, bpy, c='black', s= 1000)
    axe.scatter(wpx, wpy, c='white', s=1000)
    axe.scatter([ax, bx], [ay, by], s=1000, 
                facecolor='none', edgecolor='white' if c else 'black', linewidth=3)
    axe.scatter(cx, cy, s=200, c=R)
    axe.set_axis_bgcolor('grey')
    axe.set_xticks(np.arange(10)+.5)
    axe.set_yticks(np.arange(4) + .5)
    axe.set_xticklabels([])
    axe.set_yticklabels([])
    axe.set_ylim([-.5, 3.5])
    axe.set_xlim([-.5, 8.5])
    axe.set_aspect(1)
    return frame.error.mean()

sns.set_style('darkgrid')
f, axes = plt.subplots(1, 2, figsize=(16, 12), squeeze=False)

title1 = plotpos(hard, axe=axes[0, 0])
axes[0, 0].set_title('Performance = ' + str(title1)[:4])
title2 = plotpos(easy, axe=axes[0, 1])
axes[0, 1].set_title('Performance = ' + str(title2)[:4])
sns.despine()
f.savefig(figure_out + 'easy_hard_examples_with_markers.png', format='png')

In [ ]:
evapreds = pd.read_csv(data_out + 'eval_pred.csv')
evapreds.columns = evapreds.loc[0, :]
evapreds = evapreds.loc[1:, :].reset_index(drop=True).astype(float)

In [ ]:
sns.regplot('Observed Ranking', 'dfs', data=evapreds)

In [ ]:
for n in range(len(subjects)):
    eva.loc[:, n] = np.nan

for s1 in eva.subject.unique():
    
    for s2 in eva.subject.unique():
        
        for i in eva.loc[eva.subject == s1, :].index.values:
            bp = eva.loc[i, 'bp']
            wp = eva.loc[i, 'wp']
            r = eva.loc[i, 'response']
            condition = (eva.subject == s2) & (eva.bp == bp) & (eva.wp == wp)
           
            if len(eva.loc[condition, :]) == 1:
                eva.loc[i, s2] = eva.loc[condition, 'response'].values[0]
            elif len(eva.loc[condition, :]) > 1:
                eva.loc[i, s2] = eva.loc[condition, 'response'].values.mean()

eva.loc[:, 'mean_response'] = eva.loc[:, range(12)].mean(axis=1)
eva.loc[:, 'mean_dist_mean'] = np.abs((eva.loc[:, range(12)] - eva.loc[:, 'mean_response'])).mean(axis=1)